# Modelado Estocástico
## Clase 6 - Notación Matricial

In [3]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
import matplotlib.pyplot as plt
import numpy.linalg as nplin

In [4]:
df = pd.read_excel('ceo.xlsx', sheet_name='Hoja1', usecols='A:B')
df.head()

,Ganancias,Compensacion_CEO
0,357.0,0.7
1,48.0,0.7
2,932.0,0.8
3,366.0,0.7
4,83.0,0.8


### Regresión Compensacion_CEO ~ Ganancias (con constante)

In [5]:
X = sm.add_constant(df["Ganancias"])
y = df["Compensacion_CEO"]
model1 = sm.OLS(y, X).fit()
print(model1.summary())

                            OLS Regression Results                            
Dep. Variable:       Compensacion_CEO   R-squared:                       0.434
Model:                            OLS   Adj. R-squared:                  0.426
Method:                 Least Squares   F-statistic:                     52.24
Date:                Wed, 20 Aug 2025   Prob (F-statistic):           5.50e-10
Time:                        21:18:22   Log-Likelihood:                -73.655
No. Observations:                  70   AIC:                             151.3
Df Residuals:                      68   BIC:                             155.8
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.6000      0.112      5.342      0.0

### Vamos a realizar las cuentas operando con matrices

In [6]:
X.head()

,const,Ganancias
0,1.0,357.0
1,1.0,48.0
2,1.0,932.0
3,1.0,366.0
4,1.0,83.0


In [7]:
y.head()

0    0.7
1    0.7
2    0.8
3    0.7
4    0.8
Name: Compensacion_CEO, dtype: float64

### Matriz transpuesta de X

In [8]:
X.transpose()

,0,1,2,3,4,5,6,7,8,9,...,60,61,62,63,64,65,66,67,68,69
const,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
Ganancias,357.0,48.0,932.0,366.0,83.0,22.0,67.0,413.0,496.0,458.0,...,2304.0,29.0,2493.0,71.0,185.0,327.0,409.0,117.0,179.0,234.0


Comparamos las dimensiones de X y su matriz transpuesta:

In [9]:
print(f"X = {X.shape} \nX transpuesta = {X.transpose().shape}")

X = (70, 2) 
X transpuesta = (2, 70)


### Vamos a calcular los estimadores usando las expresiones de matrices

Sabemos que $\hat{\beta} = (X'X)^{-1}(X'y)$

`np.lininv` calcula la inversa de una matriz y `@` multiplica matrices.

Veamos dos formas de escribir lo mismo, se pueden sumar paréntesis si no cambian el sentido de las operaciones.

In [10]:
beta_sombrero = nplin.inv(X.transpose() @ X) @ (X.transpose() @ y)
beta_sombrero2 = np.linalg.inv(X.T @ X) @ (X.T @ y)

Chequeamos que son operaciones equivalentes:

In [11]:
np.allclose(beta_sombrero2, beta_sombrero)

True

### Matrices de Proyeccion

Notar que:

$e = y - \hat{y}$

$\hat{y} = X @ \hat{\beta} = X (X'X)^{-1}(X'y) = P$

donde
$P = X(X'X)^{-1}X'$

La matriz P es una matriz de $nxn$ idempotente que proyecta sobre el espacio generado por las columnas de X.

Vemos que P satisface la definición de matriz idempotente, ya que $P @ P = P$

In [12]:
y_sombrero = X @ beta_sombrero
y_sombrero.head()

0    0.900676
1    0.640397
2    1.385014
3    0.908257
4    0.669878
dtype: float64

In [13]:
model1.fittedvalues.head()

0    0.900676
1    0.640397
2    1.385014
3    0.908257
4    0.669878
dtype: float64

Como se puede apreciar, tienen los mismos valores

In [14]:
np.allclose(y_sombrero , model1.fittedvalues)

True

In [15]:
e = y - y_sombrero
e.head()

0   -0.200676
1    0.059603
2   -0.585014
3   -0.208257
4    0.130122
dtype: float64

In [16]:
model1.resid.head()

0   -0.200676
1    0.059603
2   -0.585014
3   -0.208257
4    0.130122
dtype: float64

In [17]:
np.allclose(e , model1.resid)

True

X es un dataframe de Pandas, pero queremos operar sus valores como matriz. Para eso hacemos la conversión de sus valores a NumPy:

In [18]:
X_np = X.values
P = X_np @ np.linalg.inv(X_np.T @ X_np) @ X_np.T

In [19]:
type(X)

pandas.core.frame.DataFrame

In [20]:
type(X_np)

numpy.ndarray

In [21]:
y_sombrero2 = P @ y
np.allclose(y_sombrero, y_sombrero2)

True

Tenemos otra matriz idempotente:

$M = I - P$

¿De dónde viene? De los residuos:

$e = y - \hat{y} = y - P @ y = (I-P) @ y = M @ y$

Donde la matriz identidad I tiene la misma dimension que P. Para generar la matriz identidad utilizamos `np.eye(n)`.

Notar que M es idempotente: $M @ M = M$   
($M$ es de $nxn$)

$M = I - P = I - X (X'X)^{-1}X'$

In [22]:
M = np.eye(P.shape[0]) - P

e_2 = M @ y
np.allclose(e_2, model1.resid)

True

### Cálculo del $s^2$ usando matrices:

Si recapitulamos los valores del modelo de la librería, veremos que la segunda columna es el desvío estándar del $\hat{\beta}$ correspondiente a la pendiente que multiplique la $X$ que corresponda (en regresión múltiple) o del intercepeto (fila "const")

 Estos valores son la raíz de los elementos de la diagonal principal de la matriz

$Var(\hat{\beta}) =  s^2 @ (X'X)^{-1}$

In [23]:
model1.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:       Compensacion_CEO   R-squared:                       0.434
Model:                            OLS   Adj. R-squared:                  0.426
Method:                 Least Squares   F-statistic:                     52.24
Date:                Wed, 20 Aug 2025   Prob (F-statistic):           5.50e-10
Time:                        21:18:22   Log-Likelihood:                -73.655
No. Observations:                  70   AIC:                             151.3
Df Residuals:                      68   BIC:                             155.8
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.6000      0.112      5.342      0.000       0.376       0.824
Ganancias      0.0008      0.000      7.228      0.000       0.001       0.001
==============================================================================
Omnibus:                       21.745   Durbin-Watson:                   2.045
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               36.165
Skew:                           1.136   Prob(JB):                     1.40e-08
Kurtosis:                       5.691   Cond. No.                     1.29e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.29e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [24]:
var_beta_sombrero = model1.scale * (np.linalg.inv(X.T @ X))
print(var_beta_sombrero)

[[ 1.26154123e-02 -8.68407339e-06]
 [-8.68407339e-06  1.35810239e-08]]


Verificamos que la columna `std err` es la raíz cuadrada de los elementos de la diagonal principal

In [25]:
var_beta_sombrero[0,0]**0.5

np.float64(0.11231835254287276)

In [26]:
var_beta_sombrero[1,1]**0.5

np.float64(0.00011653765029467196)

Los valores de la segunda columna del `summary` de `statsmodels` pueden accederse con `bse`, siendo el primer valor el SE del $\hat{\alpha}$ y el segundo el SE del $\hat{\beta}$

In [27]:
model1.bse

const        0.112318
Ganancias    0.000117
dtype: float64

Podemos calcular y guardar los standard errors con una sola linea:

In [28]:
se_beta = [ x**0.5 for x in np.diagonal(var_beta_sombrero)]

se_beta

[np.float64(0.11231835254287276), np.float64(0.00011653765029467196)]

In [29]:
np.allclose(model1.bse.iloc[1],se_beta[1])

True